**Data from Kaggle published by Apple on Jan 2nd (https://www.kaggle.com/prasertk/apple-product-price-list-from-26-countries-2022). Data cleaning by Fabricio Quesada.**

In [143]:
import re
import osmnx as ox
import numpy as np
import pandas as pd
import requests
from forex_python.converter import CurrencyRates
import warnings
warnings.filterwarnings('ignore')

# Data Cleaning

In [144]:
apple = pd.read_csv('apple product price list from 26 countries.csv')

**Number of countries in the file**

In [145]:
len(apple.country.unique())

26

In [146]:
apple.shape

(448, 8)

**Check data head and tail**

In [147]:
apple.head(10)

,pid,model,price,country_code,country,region,income group,scraped_date
0,0,iPhone 13,"From A$1,199",au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
1,1,iPhone SE,From A$679,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
2,2,iPhone 12,From A$999,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
3,3,AirPods(3rd generation),A$279,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
4,4,AirPods Pro,A$399,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
5,5,AirPods(2nd generation),A$219,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
6,6,AirPods Max,A$899,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
7,7,Apple TV 4K,A$249,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
8,8,Apple Watch SE,From A$429,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
9,9,Apple Watch Series 3,From A$299,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535


In [148]:
apple.tail(10)

,pid,model,price,country_code,country,region,income group,scraped_date
438,7,Apple TV 4K,From $179.00 or $29.83/mo. per month for 6 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744
439,8,Apple Watch SE,From $279 or $11.62/mo. per month for 24 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744
440,9,Apple Watch Series 3,From $199 or $8.29/mo. per month for 24 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744
441,10,Sport Band,$49 or $4.08/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744
442,11,iPad,From $329 or $27.41/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744
443,12,iPad Pro,From $799 or $66.58/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744
444,13,Apple Pencil (2nd generation),$129 or $10.75/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744
445,14,24-inch iMac²,From $1299 or $108.25/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744
446,15,MacBook Air,From $999 or $83.25/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744
447,16,Magic Mouse,$79,us,United States,North America,High income,2022-01-02 21:52:47.021744


**Spaces replaced in the price column**

In [149]:
price_list = []
for i in apple.price:
    i = i.replace(' ', '')
    price_list.append(i)
apple['price_local_currency'] = np.array(price_list)

**Find all numbers in the price column. For all products the price is always the first number, the for loop down below gets that first number**

In [150]:
p = '[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+'
d = [np.nan]*len(apple.price_local_currency)
for i in range(0, len(apple.price_local_currency)):
    e = []
    s = apple.price_local_currency[i]
    if re.search(p, s) is not None:
        for catch in re.finditer(p, s):
            e.append(catch[0])
        d[i] = e[0]
apple['price_local_currency'] = d

In [151]:
apple.head(10)

,pid,model,price,country_code,country,region,income group,scraped_date,price_local_currency
0,0,iPhone 13,"From A$1,199",au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,"1,199"
1,1,iPhone SE,From A$679,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,679
2,2,iPhone 12,From A$999,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,999
3,3,AirPods(3rd generation),A$279,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,279
4,4,AirPods Pro,A$399,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,399
5,5,AirPods(2nd generation),A$219,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,219
6,6,AirPods Max,A$899,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,899
7,7,Apple TV 4K,A$249,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,249
8,8,Apple Watch SE,From A$429,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,429
9,9,Apple Watch Series 3,From A$299,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,299


In [152]:
apple.tail(10)

,pid,model,price,country_code,country,region,income group,scraped_date,price_local_currency
438,7,Apple TV 4K,From $179.00 or $29.83/mo. per month for 6 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744,179.00
439,8,Apple Watch SE,From $279 or $11.62/mo. per month for 24 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744,279
440,9,Apple Watch Series 3,From $199 or $8.29/mo. per month for 24 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744,199
441,10,Sport Band,$49 or $4.08/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744,49
442,11,iPad,From $329 or $27.41/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744,329
443,12,iPad Pro,From $799 or $66.58/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744,799
444,13,Apple Pencil (2nd generation),$129 or $10.75/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744,129
445,14,24-inch iMac²,From $1299 or $108.25/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744,1299
446,15,MacBook Air,From $999 or $83.25/mo. per month for 12 mo.*,us,United States,North America,High income,2022-01-02 21:52:47.021744,999
447,16,Magic Mouse,$79,us,United States,North America,High income,2022-01-02 21:52:47.021744,79


**This function will help in the two dataframes merging process**

In [153]:
def special_characters (text):
        text = re.sub('á','a', text)
        text = re.sub('é','e', text)
        text = re.sub('í','i', text)
        text = re.sub('ó','o', text)
        text = re.sub('ú','u', text)
        text = re.sub('ñ','n', text)
        text = re.sub('Á','a', text)
        text = re.sub('É','e', text)
        text = re.sub('Í','i', text)
        text = re.sub('Ó','o', text)
        text = re.sub('Ú','u', text)
        text = re.sub('Ñ','N', text)
        return text.upper()

**The prices are in local currencies, they will need to be in USD.**

In [154]:
url = 'https://www.iban.com/currency-codes'
html = requests.get(url).content
df_list = pd.read_html(html)
currencies = df_list[-1]
currencies

,Country,Currency,Code,Number
0,AFGHANISTAN,Afghani,AFN,971.0
1,ÅLAND ISLANDS,Euro,EUR,978.0
2,ALBANIA,Lek,ALL,8.0
3,ALGERIA,Algerian Dinar,DZD,12.0
4,AMERICAN SAMOA,US Dollar,USD,840.0
...,...,...,...,...
263,WALLIS AND FUTUNA,CFP Franc,XPF,953.0
264,WESTERN SAHARA,Moroccan Dirham,MAD,504.0
265,YEMEN,Yemeni Rial,YER,886.0
266,ZAMBIA,Zambian Kwacha,ZMW,967.0


**To match countries between the apple table and the currency_code table I need to remove the data between parenthesis and change some countries names, for instance, 'NETHERLANDS (THE)' to 'NETHERLANDS' and 'UNITED STATES OF AMERICA' to 'UNITED STATES'**

In [155]:
for i in range(0,len(currencies['Country'])):
    currencies.Country[i] = re.sub("\(.*?\)","()",currencies.Country[i])
    currencies.Country[i] = currencies.Country[i].replace(' ()', '')
    currencies.Country[i] = currencies.Country[i].replace('UNITED STATES OF AMERICA', 'UNITED STATES')
    currencies.Country[i] = currencies.Country[i].replace('RUSSIAN FEDERATION', 'RUSSIA')

In [156]:
currencies.shape

(268, 4)

**USN and MXV appear as currencies for Mexico and United States but those won't be used. I delete them from the dataframe to avoid duplicates when merging apple table and currencies table**

In [157]:
currencies = currencies.loc[(currencies["Code"] != 'USN') & (currencies["Code"] != 'MXV')]

In [158]:
currencies.shape

(266, 4)

**Replace some additional special characters**

In [159]:
currencies.Country = [special_characters(x) for x in currencies.Country]
apple.country = [special_characters(x) for x in apple.country]

**Currencies table and apple table merge**

In [160]:
apple = apple.merge(currencies, left_on = "country", right_on = 'Country')
apple = apple.drop(['scraped_date', 'price', 'Country','Number'], axis = 1)
apple.rename(columns = {'income group': 'income_group'}, inplace=True)

In [161]:
apple.head()

,pid,model,country_code,country,region,income_group,price_local_currency,Currency,Code
0,0,iPhone 13,au,AUSTRALIA,East Asia & Pacific,High income,"1,199",Australian Dollar,AUD
1,1,iPhone SE,au,AUSTRALIA,East Asia & Pacific,High income,679,Australian Dollar,AUD
2,2,iPhone 12,au,AUSTRALIA,East Asia & Pacific,High income,999,Australian Dollar,AUD
3,3,AirPods(3rd generation),au,AUSTRALIA,East Asia & Pacific,High income,279,Australian Dollar,AUD
4,4,AirPods Pro,au,AUSTRALIA,East Asia & Pacific,High income,399,Australian Dollar,AUD


**The number of countries is the same as when I imported the data, there are not missing countries**

In [162]:
len(apple.country.unique())

26

**List of countries that use comma for decimals (https://www.dfa.cornell.edu/treasurer/cash-management/processinginternational/intl-currency)**

In [163]:
comma_countries = ['Albania',
'Algeria',
'Andorra',
'Angola',
'Argentina',
'Armenia',
'Austria',
'Azerbaijan',
'Belarus',
'Belgium',
'Bolivia',
'Bosnia and Herzegovina',
'Brazil',
'Bulgaria',
'Cameroon',
'Chile',
'Colombia',
'Costa Rica',
'Croatia',
'Cuba',
'Cyprus',
'Czechia',
'Denmark',
'East Timor',
'Ecuador',
'Estonia',
'Faroes',
'Finland',
'France',
'Germany',
'Georgia',
'Greece',
'Greenland',
'Hungary',
'Iceland',
'Indonesia',
'Italy',
'Kazakhstan',
'Kosovo',
'Kyrgyzstan',
'Latvia',
'Lebanon',
'Lithuania',
'Luxembourg',
'Macau',
'Mauritania',
'Moldova',
'Mongolia',
'Morocco',
'Mozambique',
'Namibia',
'Netherlands',
'North Macedonia',
'Norway',
'Paraguay',
'Peru',
'Poland',
'Portugal',
'Romania',
'Russia',
'Serbia',
'Slovakia',
'Slovenia',
'Somalia',
'South Africa',
'Spain',
'Suriname',
'Sweden',
'Switzerland',
'Tunisia',
'Turkey',
'Turkmenistan',
'Ukraine',
'Uruguay',
'Uzbekistan',
'Venezuela',
'Vietnam',
'Zimbabwe']

**Uppercase to all the countries in the list**

In [164]:
comma_countries = [x.upper() for x in comma_countries]
comma_countries

['ALBANIA',
 'ALGERIA',
 'ANDORRA',
 'ANGOLA',
 'ARGENTINA',
 'ARMENIA',
 'AUSTRIA',
 'AZERBAIJAN',
 'BELARUS',
 'BELGIUM',
 'BOLIVIA',
 'BOSNIA AND HERZEGOVINA',
 'BRAZIL',
 'BULGARIA',
 'CAMEROON',
 'CHILE',
 'COLOMBIA',
 'COSTA RICA',
 'CROATIA',
 'CUBA',
 'CYPRUS',
 'CZECHIA',
 'DENMARK',
 'EAST TIMOR',
 'ECUADOR',
 'ESTONIA',
 'FAROES',
 'FINLAND',
 'FRANCE',
 'GERMANY',
 'GEORGIA',
 'GREECE',
 'GREENLAND',
 'HUNGARY',
 'ICELAND',
 'INDONESIA',
 'ITALY',
 'KAZAKHSTAN',
 'KOSOVO',
 'KYRGYZSTAN',
 'LATVIA',
 'LEBANON',
 'LITHUANIA',
 'LUXEMBOURG',
 'MACAU',
 'MAURITANIA',
 'MOLDOVA',
 'MONGOLIA',
 'MOROCCO',
 'MOZAMBIQUE',
 'NAMIBIA',
 'NETHERLANDS',
 'NORTH MACEDONIA',
 'NORWAY',
 'PARAGUAY',
 'PERU',
 'POLAND',
 'PORTUGAL',
 'ROMANIA',
 'RUSSIA',
 'SERBIA',
 'SLOVAKIA',
 'SLOVENIA',
 'SOMALIA',
 'SOUTH AFRICA',
 'SPAIN',
 'SURINAME',
 'SWEDEN',
 'SWITZERLAND',
 'TUNISIA',
 'TURKEY',
 'TURKMENISTAN',
 'UKRAINE',
 'URUGUAY',
 'UZBEKISTAN',
 'VENEZUELA',
 'VIETNAM',
 'ZIMBABWE']

**Change all prices to the same format: dot for decimals**

In [165]:
for z in range(0, len(apple.country)):
    if apple.country[z] not in comma_countries:
        apple.price_local_currency[z] = str(apple.price_local_currency[z]).replace(',', '')
    else:
        apple.price_local_currency[z] = str(apple.price_local_currency[z]).replace('.', '')
        apple.price_local_currency[z] = str(apple.price_local_currency[z]).replace(',', '.')

**The shape remains the same**

In [166]:
apple.shape

(448, 9)

**Check for nan values**

In [167]:
apple[apple["price_local_currency"] == 'nan']

,pid,model,country_code,country,region,income_group,price_local_currency,Currency,Code
127,8,Tarjeta regalo del Apple Store,es,SPAIN,Europe & Central Asia,High income,nan,Euro,EUR
231,8,Carta regalo Apple Store,it,ITALY,Europe & Central Asia,High income,nan,Euro,EUR
266,8,Tarjetas App Store & iTunes,mx,MEXICO,Latin America & Caribbean,Upper middle income,nan,Mexican Peso,MXN
369,8,Подарочные карты iTunes,ru,RUSSIA,Europe & Central Asia,Upper middle income,nan,Russian Ruble,RUB
421,8,App Store ve iTunes Hediye Kartı satın alın,tr,TURKEY,Europe & Central Asia,Upper middle income,nan,Turkish Lira,TRY


**Drop nan values**

In [168]:
apple = apple.loc[(apple["price_local_currency"] != 'nan')]

**Change column to numeric**

In [169]:
apple[["price_local_currency"]] = apple[["price_local_currency"]] .apply(pd.to_numeric)

**Prices in USD (Currency Exchange)**

In [170]:
dicte = {}
cur = apple.Code.unique()
c = CurrencyRates()
for l in range(0, len(cur)):
     dicte.update({cur[l] : c.convert( cur[l], 'USD', 1)})

In [171]:
rates = pd.DataFrame(
    {'codes': dicte.keys(), 'rate': dicte.values()})
rates

,codes,rate
0,AUD,0.723663
1,EUR,1.135500
2,BRL,0.178709
3,CAD,0.787448
4,CZK,0.045753
5,DKK,0.152658
6,HUF,0.003088
7,INR,0.013455
8,MXN,0.048889
9,NOK,0.113535


**Merge apple table with rates table**

In [172]:
apple = apple.merge(rates, left_on = "Code", right_on = 'codes')

**Get prices in USD**

In [173]:
try:
    apple['price_usd'] = apple['price_local_currency']*apple['rate']
except KeyError as e:
    raise ValueError('failed') from e

**Model column needs to be in english**

In [174]:
apple['model_en'] = ''

for m in range (0, len(apple.model)):
    if "iMac" in apple.model[m]:
        apple['model_en'][m] = '24-inch iMac²'
    else:
        apple['model_en'][m] = apple.model[m]
        
for n in range (0, len(apple.model)):
    if "AirPods(2" in apple.model[n]:
        apple['model_en'][n] = 'AirPods(2nd generation)'
    else:
        apple['model_en'][n] = apple.model_en[n]
        
for o in range (0, len(apple.model)):
    if "AirPods(3" in apple.model[o]:
        apple['model_en'][o] = 'AirPods(3rd generation)'
    else:
        apple['model_en'][o] = apple.model_en[o]
        
for p in range (0, len(apple.model)):
    if "Apple Pencil" in apple.model[p]:
        apple['model_en'][p] = 'Apple Pencil (2nd generation)'
    else:
        apple['model_en'][p] = apple.model_en[p]

In [175]:
apple.model_en.unique()

array(['iPhone 13', 'iPhone SE', 'iPhone 12', 'AirPods(3rd generation)',
       'AirPods Pro', 'AirPods(2nd generation)', 'AirPods Max',
       'Apple TV 4K', 'Apple Watch SE', 'Apple Watch Series 3',
       'Sport Band', 'iPad', 'iPad Pro', 'Apple Pencil (2nd generation)',
       '24-inch iMac²', 'MacBook Air', 'Magic Mouse', 'Apple TV HD',
       'Sportarmband', 'iPhone 13 Pro', 'Correa deportiva',
       'Urheiluranneke', 'Bracelet Sport', 'AirPods(terza generazione)',
       'AirPods(seconda generazione)', 'Cinturino Sport', 'Sportbandje',
       'Bracelete desportiva', 'Pulseira esportiva', 'Sportovní řemínek',
       'Sportsrem', 'Sportszíj', 'AirPods(tercera generación)',
       'AirPods(segunda generación)', 'Pasek sportowy',
       'Спортивный ремешок', 'Мышь Magic Mouse',
       'AirPods(tredje generationen)', 'AirPods(andra generationen)',
       'Sportband', 'AirPods(รุ่นที่ 3)', 'AirPods(รุ่นที่ 2)',
       'สายแบบ Sport Band', 'Spor Kordon'], dtype=object)

In [176]:
translate = {'AirPods(terza generazione)':'AirPods(3rd generation)',
             'AirPods(seconda generazione)':'AirPods(2nd generation)',
             'AirPods(tercera generación)':'AirPods(3rd generation)',
             'AirPods(segunda generación)':'AirPods(2nd generation)',
             'AirPods(tredje generationen)':'AirPods(3rd generation)',
             'AirPods(andra generationen)': 'AirPods(2nd generation)',
             'AirPods(รุ่นที่ 3)': 'AirPods(3rd generation)', 'AirPods(รุ่นที่ 2)':'AirPods(2nd generation)',
             'Мышь Magic Mouse':'Magic Mouse', 'Sportarmband':'Sport Band', 
             'Correa deportiva':'Sport Band', 'Urheiluranneke':'Sport Band',
             'Bracelet Sport':'Sport Band','Cinturino Sport':'Sport Band',
             'Sportbandje':'Sport Band','Bracelete desportiva':'Sport Band','Pulseira esportiva':'Sport Band',
             'Sportovní řemínek':'Sport Band','Sportsrem':'Sport Band','Sportszíj':'Sport Band',
             'Pasek sportowy':'Sport Band', 'Спортивный ремешок': 'Sport Band',
             'สายแบบ Sport Band':'Sport Band','Spor Kordon':'Sport Band'}

In [177]:
for q in range (0, len(apple.model_en)):
    for r in range(0,len(list(translate.keys()))):
        if list(translate.keys())[r] in apple.model_en[q]:
            apple['model_en'][q] = list(translate.values())[r]
        else:
            apple['model_en'][q] = apple.model_en[q]

**Some final formatting**

In [178]:
apple = apple[['pid', 'model_en', 'country', 'region', 'income_group', 'price_local_currency', 'Code', 'price_usd']]
apple.rename(columns = {'Code': 'code','model_en':'model'}, inplace=True)
apple['price_usd'] = round(apple['price_usd'],2)
for u in range(0, len(apple.country)):
    apple.country[u] = apple.country[u].title()
apple.head()

,pid,model,country,region,income_group,price_local_currency,code,price_usd
0,0,iPhone 13,Australia,East Asia & Pacific,High income,1199.0,AUD,867.67
1,1,iPhone SE,Australia,East Asia & Pacific,High income,679.0,AUD,491.37
2,2,iPhone 12,Australia,East Asia & Pacific,High income,999.0,AUD,722.94
3,3,AirPods(3rd generation),Australia,East Asia & Pacific,High income,279.0,AUD,201.90
4,4,AirPods Pro,Australia,East Asia & Pacific,High income,399.0,AUD,288.74


**Save new file**

In [179]:
apple.to_csv('apple.csv')